## Clasificación TF-IDF 

In [39]:
import pandas as pd
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess

Lo primero que necesitamos es cargar nuestros glosarios de términos para crear nuestro diccionario

In [42]:
def cargar_glosario(categoria):
    fname = f"../Datos/glosario_{categoria}.txt"
    glosario = []
    with open(fname, 'r') as f:
        glosario = [termino.rstrip('\n') for termino in f.readlines()]
    return glosario

In [ ]:
categorias = ["deportes", "salud", "ciencia", "politica"]
glosarios = {}
for categoria in categorias:
    glosarios[categoria] = cargar_glosario(categoria)
glosarios

Creamos nuestro diccionario de palabras en base a los terminos de todos los glosarios

In [ ]:
def create_dictionary(glosarios):
    doc_tokens = [[termino for termino in glosario] for glosario in glosarios.values()]
    dictionary = corpora.Dictionary(doc_tokens)
    return dictionary

In [ ]:
glosarios_dict = create_dictionary(glosarios)
print(glosarios_dict)

Dictionary<140 unique tokens: ['arabia', 'booker', 'butler', 'campazzo', 'carpena']...>


Cargamos nuestras noticias de test y las convertimos a un bag of words utilizando nuestro diccionario

In [ ]:
noticias_test_dataframe = pd.read_csv("../Datos/noticias_test.csv")

In [ ]:
def create_bag_of_words(docs_list, dictionary):
    doc_tokens = [simple_preprocess(corpus) for corpus in docs_list]
    docs_bow = [dictionary.doc2bow(doc) for doc in doc_tokens]
    return docs_bow

In [ ]:
def create_tfidf(docs_bow):
    docs_tfidf = models.TfidfModel(docs_bow, smartirs="lfc")
    return docs_tfidf

Ahora es momento de crear una bag of words para cada noticia en base a nuestro diccionario

In [ ]:
docs_bow = create_bag_of_words(noticias_test_dataframe["corpus"].values, glosarios_dict)
print(docs_bow[19])

[(93, 2)]


Y con el bag of words podemos crear nuestro modelo tfidf

In [ ]:
tfidf_model = create_tfidf(docs_bow)
tfidf_model[docs_bow[0]]

[(44, 0.6103543701830407), (137, 0.7921284888188681)]

Representamos nuestros glosarios en tfidf

In [ ]:
deportes_bow = glosarios_dict.doc2bow(glosarios["deportes"])
deportes_tfidf = tfidf_model[deportes_bow]

salud_bow = glosarios_dict.doc2bow(glosarios["salud"])
salud_tfidf = tfidf_model[salud_bow]

ciencia_bow = glosarios_dict.doc2bow(glosarios["ciencia"])
ciencia_tfidf = tfidf_model[ciencia_bow]

politica_bow = glosarios_dict.doc2bow(glosarios["politica"])
politica_tfidf = tfidf_model[politica_bow]

Ahora clasificamos

In [ ]:
index = similarities.MatrixSimilarity(tfidf_model[docs_bow])
similarities_deportes = index[deportes_tfidf]
similarities_salud = index[salud_tfidf]
similarities_ciencia = index[ciencia_tfidf]
similarities_politica = index[politica_tfidf]

In [ ]:
def get_prediction(index, 
                   similarities_deportes,       
                   similarities_salud,                       
                   similarities_ciencia,
                   similarities_politica):
    mejor_match = 0
    mejor = "No clasificado"
    if (similarities_deportes[index] > mejor_match):
        mejor_match = similarities_deportes[index]
        mejor = "sports"
    if (similarities_salud[index] > mejor_match):
        mejor_match = similarities_salud[index]
        mejor = "health"
    if (similarities_ciencia[index] > mejor_match):
        mejor_match = similarities_ciencia[index]
        mejor = "science"
    if (similarities_politica[index] > mejor_match):
        mejor_match = similarities_politica[index]
        mejor = "politics"
    return mejor
    

In [ ]:
predictions = []
for i in range(len(docs_bow)):
    pred = get_prediction(i, 
                          similarities_deportes,
                          similarities_salud,
                          similarities_ciencia,
                          similarities_politica)
    predictions.append(pred)

In [ ]:
noticias_test_dataframe.insert(2, "labels", predictions)

In [ ]:
noticias_test_dataframe.to_csv('../Prediccion/predicciones.csv')

In [ ]:
display(noticias_test_dataframe)

,index,category,labels,n_doc,title,path,link,docs,corpus
0,32,sports,politics,33,Stakhovsky carga contra los tenistas serbios p...,./Datos/Raw_data/sports/33.txt,https://news.google.com/__i/rss/rd/articles/CB...,El extenista ucraniano Sergiy Stakhovsky sigue...,extenista ucraniano Sergiy Stakhovsky cruzada ...
1,6,sports,politics,7,"El último baile del invicto Van Gaal: ""Luchamo...",./Datos/Raw_data/sports/7.txt,https://news.google.com/__i/rss/rd/articles/CB...,"El seleccionador de Países Bajos, que encadena...",seleccionador Paises Bajos encadenar 19 partid...
2,44,sports,sports,45,El olor caro de Nadal - Noticias de Gipuzkoa,./Datos/Raw_data/sports/45.txt,https://news.google.com/__i/rss/rd/articles/CB...,Lo ha ganado absolutamente todo dentro de las ...,ganar absolutamente pista tenis Abiertos Austr...
3,19,sports,sports,20,T.J. Warren vuelve vuelve a las canchas tras 7...,./Datos/Raw_data/sports/20.txt,https://news.google.com/__i/rss/rd/articles/CB...,La pesadilla de T.J. Warren tuvo anoche su fin...,pesadilla T. J. Warren anoche feliz 24 mes pis...
4,59,sports,sports,60,Ferrari aclara las teorías sobre su degradació...,./Datos/Raw_data/sports/60.txt,https://news.google.com/__i/rss/rd/articles/CB...,El equipo de Maranello comenzÃ³ el aÃ±o siendo...,equipo Maranello comenzA3 aAo rival batir Char...
...,...,...,...,...,...,...,...,...,...
115,185,politics,sports,6,Que el ruido no ensordezca lo importante,./Datos/Raw_data/politics/6.txt,https://www.20minutos.es/noticia/5083175/0/pso...,e Quienes ganen mÃ¡s de 250.000 euros extraord...,ganar mas 250 000 euros extraordinario justifi...
116,228,politics,sports,49,Sandro Rosell anunciará tras Reyes si sepresen...,./Datos/Raw_data/politics/49.txt,https://elpais.com/espana/madrid/2022-12-12/al...,Javier Ortega Smith lleva un tatuaje en un bra...,Javier Ortega Smith tatuaje brazo poner COE 13...
117,218,politics,sports,39,"Feijóo rechaza la moción de censura deVox, per...",./Datos/Raw_data/politics/39.txt,https://www.20minutos.es/noticia/5084012/0/num...,e TransiciÃ³n EcolÃ³gica se pone en contacto c...,TransiciA3n ecola3gica poner contacto administ...
118,230,politics,politics,51,El H2Med costará 2.500 millones para llevarun ...,./Datos/Raw_data/politics/51.txt,https://www.eldiario.es/politica/gobierno-aseg...,âLa agenda polÃ­tica de ERC es la suya y es ...,ala agenda polatico ERC contrario ConstituciA3...
